In [9]:
from joblib import dump, load
import os
import pickle
import argparse
# For text preprocessing
import re
import numpy as np
from nltk.tokenize import TweetTokenizer
from wordsegment import segment, load
import sys
from sklearn.model_selection import train_test_split, KFold
from datasets import load_dataset
import pandas as pd
from tqdm.auto import tqdm
import pickle
import os

import os
import sys
import numpy as np
import argparse
import pickle
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline

from joblib import dump, load

import warnings
from sklearn.metrics import classification_report

import pandas as pd
import functools

sys.path.insert(1, os.path.join(sys.path[0], '..'))
# import helper
import time

import argparse
import re

def train(classifier,n_gram_tuple,feature_level,max_feature_length,_data,labels):
    if classifier == 'NB':
        text_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(n_gram_tuple[0],n_gram_tuple[1]), analyzer=feature_level, max_features=max_feature_length)),
                        ('clf', MultinomialNB())])

    elif classifier == 'LR':
        text_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(n_gram_tuple[0],n_gram_tuple[1]), analyzer=feature_level, max_features=max_feature_length)),
                        ('clf', LogisticRegression(multi_class="multinomial", solver="lbfgs"))])

    elif classifier == 'SVM':
        text_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(n_gram_tuple[0],n_gram_tuple[1]), analyzer=feature_level, max_features=max_feature_length)),
                        ('clf', SGDClassifier(loss='log', penalty='l2'))])

    elif classifier == 'RF':
        text_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(n_gram_tuple[0],n_gram_tuple[1]), analyzer=feature_level, max_features=max_feature_length)),
                        ('clf', RandomForestClassifier())]) 

    elif classifier == 'GBT':
        text_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(n_gram_tuple[0],n_gram_tuple[1]), analyzer=feature_level, max_features=max_feature_length)),
                        ('clf', GradientBoostingClassifier(learning_rate=1, max_depth=1))])
        
    
    train_label_data = [i.tolist().index(1) for i in _data['label_train']]
    text_clf.fit(_data["text_train"], train_label_data)
    
    preds = text_clf.predict(_data["text_test"])
    
    test_label_data = [i.tolist().index(1) for i in _data['label_test']]

    print(classifier)
#     print(labels)
    print(classification_report(test_label_data, preds,target_names=labels))
    print("======================")
    
    return text_clf,test_label_data,preds


In [10]:
models = ['NB','LR','SVM','RF','GBT']
languages = ['english_1','filipino','chinese','korean']

In [11]:
# import os
# import pickle

# Word Level

In [12]:
args = {'feature_level':'word',
        'clf':'LR',
        'language':None,
        'ngram_range':(1,3),
        'max_features':14000,
        
       }

for lang in languages:
    args['language'] = lang

    print(args['language'])
    for m in tqdm(models):

        with open(f"processed_data/{args['language']}_processed.pkl", "rb") as f:
            _data = pickle.load(f)

        clf = train(m,args['ngram_range'],args['feature_level'],args['max_features'],_data,list(_data['label_dict'].keys()))

        model_name = args['language']+'_'+m+'_'+args["feature_level"]+'.joblib'
        dump(clf, f'output/{model_name}') 


english_1


  0%|          | 0/5 [00:00<?, ?it/s]

NB
               precision    recall  f1-score   support

    offensive       0.88      1.00      0.93      4124
not_offensive       0.93      0.32      0.47       833

     accuracy                           0.88      4957
    macro avg       0.90      0.66      0.70      4957
 weighted avg       0.89      0.88      0.86      4957

LR
               precision    recall  f1-score   support

    offensive       0.95      0.98      0.96      4124
not_offensive       0.88      0.73      0.80       833

     accuracy                           0.94      4957
    macro avg       0.92      0.86      0.88      4957
 weighted avg       0.94      0.94      0.94      4957

SVM
               precision    recall  f1-score   support

    offensive       0.91      0.99      0.95      4124
not_offensive       0.90      0.54      0.68       833

     accuracy                           0.91      4957
    macro avg       0.91      0.76      0.81      4957
 weighted avg       0.91      0.91      0.90   

  0%|          | 0/5 [00:00<?, ?it/s]

NB
               precision    recall  f1-score   support

    offensive       0.73      0.78      0.75      2251
not_offensive       0.79      0.75      0.77      2596

     accuracy                           0.76      4847
    macro avg       0.76      0.76      0.76      4847
 weighted avg       0.76      0.76      0.76      4847

LR
               precision    recall  f1-score   support

    offensive       0.82      0.79      0.80      2251
not_offensive       0.82      0.85      0.84      2596

     accuracy                           0.82      4847
    macro avg       0.82      0.82      0.82      4847
 weighted avg       0.82      0.82      0.82      4847

SVM
               precision    recall  f1-score   support

    offensive       0.79      0.75      0.77      2251
not_offensive       0.79      0.82      0.81      2596

     accuracy                           0.79      4847
    macro avg       0.79      0.79      0.79      4847
 weighted avg       0.79      0.79      0.79   

  0%|          | 0/5 [00:00<?, ?it/s]

NB
               precision    recall  f1-score   support

    offensive       0.62      0.03      0.06       619
not_offensive       0.66      0.99      0.79      1175

     accuracy                           0.66      1794
    macro avg       0.64      0.51      0.42      1794
 weighted avg       0.65      0.66      0.54      1794

LR
               precision    recall  f1-score   support

    offensive       0.46      0.06      0.10       619
not_offensive       0.66      0.97      0.78      1175

     accuracy                           0.65      1794
    macro avg       0.56      0.51      0.44      1794
 weighted avg       0.59      0.65      0.55      1794

SVM
               precision    recall  f1-score   support

    offensive       0.44      0.05      0.08       619
not_offensive       0.66      0.97      0.78      1175

     accuracy                           0.65      1794
    macro avg       0.55      0.51      0.43      1794
 weighted avg       0.58      0.65      0.54   

  0%|          | 0/5 [00:00<?, ?it/s]

NB
               precision    recall  f1-score   support

not_offensive       0.73      0.39      0.51       697
    offensive       0.66      0.89      0.75       903

     accuracy                           0.67      1600
    macro avg       0.69      0.64      0.63      1600
 weighted avg       0.69      0.67      0.65      1600

LR
               precision    recall  f1-score   support

not_offensive       0.65      0.50      0.56       697
    offensive       0.67      0.80      0.73       903

     accuracy                           0.67      1600
    macro avg       0.66      0.65      0.65      1600
 weighted avg       0.66      0.67      0.66      1600

SVM
               precision    recall  f1-score   support

not_offensive       0.67      0.48      0.56       697
    offensive       0.67      0.82      0.74       903

     accuracy                           0.67      1600
    macro avg       0.67      0.65      0.65      1600
 weighted avg       0.67      0.67      0.66   

# Char Level

In [ ]:
args = {'feature_level':'char',
        'clf':'LR',
        'language':None,
        'ngram_range':(1,3),
        'max_features':53000,
        
       }

for lang in languages:
    args['language'] = lang

    print(args['language'])
    for m in tqdm(models):

        with open(f"processed_data/{args['language']}_processed.pkl", "rb") as f:
            _data = pickle.load(f)

        clf = train(m,args['ngram_range'],args['feature_level'],args['max_features'],_data,list(_data['label_dict'].keys()))

        model_name = args['language']+'_'+m+'_'+args["feature_level"]+'.joblib'
        dump(clf, f'output/{model_name}') 


english_1


  0%|          | 0/5 [00:00<?, ?it/s]

NB
               precision    recall  f1-score   support

    offensive       0.86      1.00      0.92      4124
not_offensive       0.88      0.17      0.28       833

     accuracy                           0.86      4957
    macro avg       0.87      0.58      0.60      4957
 weighted avg       0.86      0.86      0.81      4957

LR
               precision    recall  f1-score   support

    offensive       0.95      0.98      0.96      4124
not_offensive       0.87      0.76      0.81       833

     accuracy                           0.94      4957
    macro avg       0.91      0.87      0.89      4957
 weighted avg       0.94      0.94      0.94      4957



# Next